In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact
import ipywidgets as wdgt
from IPython.display import display

In [2]:
def sys(y, t, u, T, dz, k):
    y1, y2 = y
    dydt = [y2, - (T**2)*y2 - 2*T*dz*y1 + k*u]
    return dydt

def PID (e, Kp, Ti, ei, Td, de, umin, umax, dt):
    u = Kp * (e + (1/Ti)*ei + Td*de)
    if u > umax:
        u = umax
        ei = ei - e*dt
    if u < umin:
        u = umin
        eu = ei - e*dt
    return u, ei

In [18]:
def Sys (T, dz, k):
    t = np.linspace(0, 10, 101)
    y = np.zeros((len(t)))
    u = np.zeros(len(t))
    u[10:] = 1
    y0 = [0, 0]

    for i in range(len(t)-1):
        ts = [t[i], t[i+1]]
        sol = odeint(sys, y0, ts, args = (u[i], T, dz, k))
        y0 = sol[-1,:]
        y[i+1] = y0[0]
    plt.plot(t, y, label='y(t)')
    plt.plot(t, u, label='sp(t)')
    plt.grid()
    plt.legend()
    plt.xlabel('t') 
    plt.title('Step response')

In [19]:
Ts = wdgt.FloatSlider(min = 0, max = 10, step = 1, value = 1)
dzs = wdgt.FloatSlider(min = 0, max = 10, step = .1, value = 6)
ks = wdgt.FloatSlider(min = 0, max = 10, step = 1, value = 4)
wdgt.interact(Sys, T = Ts, dz = dzs, k = ks)

interactive(children=(FloatSlider(value=1.0, description='T', max=10.0, step=1.0), FloatSlider(value=6.0, desc…

<function __main__.Sys(T, dz, k)>

In [22]:
def Sym (Kp, Ti, Td):
    T = Ts.value
    dz = dzs.value
    k = ks.value
    
    t = np.linspace(0, 50, 501)
    dt = t[1] - t[0]
    u = np.zeros(len(t))
    sp = np.zeros(len(t))
    sp[10:] = 1
    
    y = np.zeros((len(t), 2))
    e = np.zeros(len(t))
    ei = np.zeros(len(t))
    de = np.zeros(len(t))
    
    for i in range (len(t)-1):
        ts = [t[i], t[i+1]]
        e[i] = sp[i] - y[i,0]
        if i >=1:
            ei[i] = ei[i-1] + e[i]*dt
            de[i] = (y[i,0] - y[i-1,0])/dt
            u[i], ei[i] = PID(e[i], Kp, Ti, ei[i], Td, de[i], 0, 3, dt) 

        sol = odeint(sys, y[i], ts, args = (u[i], T, dz, k))
        y[i+1,0] = sol[-1,0]

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    ax1.plot(t[:-1], y[:-1,0], label='y(t)')
    ax1.plot(t, sp, label='sp(t)')
    ax2.plot(t[:-1], u[:-1], label='u(t)')
    ax1.grid()
    ax2.grid()
    ax1.legend()
    ax2.legend()  
    ax1.set_xlabel('t')
    ax2.set_xlabel('t')
    ax1.set_title('Step response')
    ax2.set_title('Control signal')

In [21]:
Kps = wdgt.FloatSlider(min = 1, max = 10, step = 1, value = 2)
Tis = wdgt.FloatSlider(min = .1, max = 10, step = .1, value = 1)
Tds = wdgt.FloatSlider(min = 0, max = 10, step = .1, value = .1)
wdgt.interact_manual(Sym, Kp = Kps, Ti = Tis, Td = Tds)

interactive(children=(FloatSlider(value=2.0, description='Kp', max=10.0, min=1.0, step=1.0), FloatSlider(value…

<function __main__.Sym(Kp, Ti, Td)>